In [1]:
# Librairies
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pickleshare import PickleShareDB

from imblearn.over_sampling import SMOTE

In [2]:
# Pour garantir la reproductibilité
np.random.seed(42)  # Pour numpy

## Chargement des données
Nous chargeons les données depuis le fichier des données PCA.

In [3]:
# Données nettoyées
db = PickleShareDB('../prep_data/kity')

sample_df = db['pca_result_with_labels']

In [4]:
sample_df['label'].value_counts()

label
normal            3174002
DoS               1290386
MITM               538852
physical fault     387126
anomaly                97
scan                   15
Name: count, dtype: int64

Le type d'attaque "scan" possède très peu d'entrées, ce qui empêche les modèles de s'entraîner correctement. Nous supprimons donc ce type d'attaque.

In [5]:
to_drop = sample_df[sample_df['label'] == 'scan'].index
sample_df = sample_df.drop(to_drop)

Nous avons préparé les données pour les classifications en séparant les caractéristiques et les étiquettes. Les données ont ensuite été divisées en ensembles d'entraînement et de test, avec une stratification pour préserver la distribution des classes. Enfin, un oversampling avec SMOTE a été appliqué sur l'ensemble d'entraînement pour équilibrer les classes, en augmentant artificiellement la représentation de la classe minoritaire (attaques).

## Reduction de la taille du dataset
Nous allons utiliser un dataset échantillon représentant 10 % du dataset original.  
Cela permet de conserver une partie significative des données sans que le dataset soit considéré comme volumineux, ce qui serait coûteux en termes de calcul et de mémoire.  
Nous effectuons un échantillonnage tout en respectant la distribution du dataset complet.

In [6]:
sample_df = sample_df.groupby('label', group_keys=False).apply(
    lambda x: x.sample(frac=0.1, random_state=42)
)

/tmp/ipykernel_157361/3490013237.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = sample_df.groupby('label', group_keys=False).apply(


In [7]:
sample_df['label'].value_counts()

label
normal            317400
DoS               129039
MITM               53885
physical fault     38713
anomaly               10
Name: count, dtype: int64

In [8]:
# Séparation des étiquettes et suppression des colonnes correspondantes du DataFrame
labels = sample_df[['label', 'label_n']]
sample_df.drop(columns=['label', 'label_n'], inplace=True)

## Preparation pour la classification binaire

In [9]:
# Les données d'entrée (X) et les étiquettes (y) pour la classification binaire
X = sample_df
y_label_n = labels['label_n']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y_label_n, test_size=0.2, random_state=42, stratify=y_label_n
)

# Équilibrage des classes via SMOTE (oversampling des classes minoritaires)
smote = SMOTE(sampling_strategy='minority')
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [10]:
# Get unique values and their counts
unique_values, counts = np.unique(y_train_resampled, return_counts=True)

# Display the result
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

Value: False, Count: 253920
Value: True, Count: 253920


## Enregistrement dans PickleShare

In [11]:
# Sauvegarde des données de classification binaire dans PickleShareDB
db['binary_X_train_resampled'] = X_train_resampled
db['binary_y_train_resampled'] = y_train_resampled
db['binary_X_test'] = X_test
db['binary_y_test'] = y_test

## Preparation pour la classification multiclasse

In [12]:
# Les étiquettes pour la classification multi-classes
y_label = labels['label']

# Liste ordonnée des classes (pour garantir un encodage cohérent des labels)
ordered_classes = ['normal', 'DoS', 'physical fault', 'MITM', 'anomaly']

# Encodage des étiquettes avec LabelEncoder
le = LabelEncoder()
le.classes_ = np.array(ordered_classes)
label_mapping_network = {label: encoded for label, encoded in zip(le.classes_, range(len(le.classes_)))}
y_label_encoded = le.transform(y_label)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y_label_encoded, test_size=0.2, random_state=42, stratify=y_label_encoded
)

# Équilibrage des classes via SMOTE (oversampling des classes minoritaires)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Enregistrement dans PickleShare

In [13]:
# Sauvegarde des données de classification multi-classes dans PickleShareDB
db['multiclass_X_train_resampled'] = X_train_resampled
db['multiclass_y_train_resampled'] = y_train_resampled
db['multiclass_X_test'] = X_test
db['multiclass_y_test'] = y_test
db['multiclass_label_mapping'] = label_mapping_network

In [14]:
db['label_mapping_network'] = label_mapping_network